# Regularization

Let's improve our understanding of what impacted **Titanic** passengers' chance of survival
- We will use logistic classifiers which are easy to interpret
- Remember we already did it with statsmodels in lecture "Decision Science - Logistic Regression"
- We were using `p-values` & statistical assumptions to detect which features were irrelevant / don't generalize
- This time, we will use `regularization` to detect relevant/irrelevant features based on under/overfitting criteria
- **Our goal is to compare `L1` and `L2` penalties**

## 1. We load and preprocess the data for you

In [8]:
import pandas as pd
import numpy as np
from sklearn.linear_model import Ridge, Lasso, LogisticRegression
from sklearn.inspection import permutation_importance
from sklearn.model_selection import train_test_split

In [2]:
data = pd.read_csv("https://wagon-public-datasets.s3.amazonaws.com/Machine%20Learning%20Datasets/ML_titanic_dataset_encoded.csv")

# the dataset is already one-hot-encoded
data.head()

,survived,pclass,age,sibsp,parch,fare,sex_female,class_First,class_Third,who_child,embark_town_Cherbourg,embark_town_Queenstown,embark_town_Southampton
0,0,3,22.0,1,0,7.2500,0,0,1,0,0,0,1
1,1,1,38.0,1,0,71.2833,1,1,0,0,1,0,0
2,1,3,26.0,0,0,7.9250,1,0,1,0,0,0,1
3,1,1,35.0,1,0,53.1000,1,1,0,0,0,0,1
4,0,3,35.0,0,0,8.0500,0,0,1,0,0,0,1


In [3]:
# We build X and y

y = data["survived"]
X = data.drop(columns="survived")
X.head()

,pclass,age,sibsp,parch,fare,sex_female,class_First,class_Third,who_child,embark_town_Cherbourg,embark_town_Queenstown,embark_town_Southampton
0,3,22.0,1,0,7.2500,0,0,1,0,0,0,1
1,1,38.0,1,0,71.2833,1,1,0,0,1,0,0
2,3,26.0,0,0,7.9250,1,0,1,0,0,0,1
3,1,35.0,1,0,53.1000,1,1,0,0,0,0,1
4,3,35.0,0,0,8.0500,0,0,1,0,0,0,1


In [5]:
# We MinMaxScale our features for you
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler().fit(X)
X_scaled = scaler.transform(X)
X_scaled = pd.DataFrame(X_scaled, columns=X.columns)
X_scaled.head()

,pclass,age,sibsp,parch,fare,sex_female,class_First,class_Third,who_child,embark_town_Cherbourg,embark_town_Queenstown,embark_town_Southampton
0,1.0,0.271174,0.2,0.0,0.014151,0.0,0.0,1.0,0.0,0.0,0.0,1.0
1,0.0,0.472229,0.2,0.0,0.139136,1.0,1.0,0.0,0.0,1.0,0.0,0.0
2,1.0,0.321438,0.0,0.0,0.015469,1.0,0.0,1.0,0.0,0.0,0.0,1.0
3,0.0,0.434531,0.2,0.0,0.103644,1.0,1.0,0.0,0.0,0.0,0.0,1.0
4,1.0,0.434531,0.0,0.0,0.015713,0.0,0.0,1.0,0.0,0.0,0.0,1.0


## 1.  Logistic Regression without regularization

❓ Rank the features by decreasing order of importance after training a simple **non-regularized** Logistic Regression (i.e. look at the coefficients after fitting)
- Careful, `LogisticRegression` is penalized by default
- Increase `max_iter` to a larger number until model converges

In [28]:
# Fit model
X_train, X_test, y_train, y_test = train_test_split(X_scaled,y,test_size=0.3)

model = LogisticRegression(penalty='none', max_iter=1000).fit(X_train, y_train) 
model.coef_


array([[  2.31358784,  -1.4910257 ,  -2.56373737,  -1.14637164,
          2.67659107,   2.70138555,   1.83384079,  -2.47405163,
          1.61814153, -10.61295061, -10.87801913, -10.67539761]])

In [29]:
coef = pd.DataFrame(model.coef_, columns=X.columns)
coef

,pclass,age,sibsp,parch,fare,sex_female,class_First,class_Third,who_child,embark_town_Cherbourg,embark_town_Queenstown,embark_town_Southampton
0,2.313588,-1.491026,-2.563737,-1.146372,2.676591,2.701386,1.833841,-2.474052,1.618142,-10.612951,-10.878019,-10.675398


❓How do you interpret, in plain English, the value for the coefficient `sex_female`?

<details>
    <summary>Answer</summary>

> "All other things being equal (such as age, ticket class etc...),
being a women increases your log-odds of survival by 2.67 (your coef value)"
    
> "Controling for all other explaining factors available in this dataset,
being a women increases your odds of survival by exp(2.67) = 14"

</details>


❓ What is the feature that most impacts the chances of survival according to your model?  
Fill the `top_1_feature` list below with the name of this feature

In [26]:
top_1_feature = ["embark_town_Southampton"]

In [27]:
from nbresult import ChallengeResult
result = ChallengeResult('unregularized', top_1_feature = top_1_feature)
result.write()
print(result.check())


============================= test session starts ==============================
platform linux -- Python 3.10.6, pytest-7.1.3, pluggy-1.0.0 -- /home/leomichel/.pyenv/versions/lewagon/bin/python3
cachedir: .pytest_cache
rootdir: /home/leomichel/code/LeonMichel96/data-regularization/tests
plugins: anyio-3.6.2, asyncio-0.19.0
asyncio: mode=strict
collecting ... collected 1 item

test_unregularized.py::TestUnregularized::test_top_1 PASSED              [100%]

============================== 1 passed in 0.03s ===============================


💯 You can commit your code:

git add tests/unregularized.pickle

git commit -m 'Completed unregularized step'

git push origin master



## 2.  Logistic Regression with an L2 penalty

Let's use a **Logistic model** whose log-loss has been penalized with a **L2** term to figure out the **most important features** without overfitting.  
This is the "classification" equivalent to the "Ridge" regressor

❓ Instantiate a **strongly regularized** `LogisticRegression` and rank its features by importance (look at the coefficients)
- By "strongly regularized" we mean "more than Sklearn's default regularization factor". 
- Sklearn's default values are very useful orders of magnitudes to keep in mind for "scaled features"

In [30]:
model_2 = LogisticRegression(penalty='l2', max_iter=1000, solver='lbfgs').fit(X_train, y_train)


In [31]:
model_2.coef_

array([[-0.66600665, -0.98374689, -1.40498029, -0.46921596,  0.74862166,
         2.46784359,  0.42688175, -0.90580963,  1.1877049 ,  0.08111614,
        -0.17917292, -0.12389603]])

In [33]:
coef2 = pd.DataFrame(model_2.coef_, columns=X.columns)
coef2

,pclass,age,sibsp,parch,fare,sex_female,class_First,class_Third,who_child,embark_town_Cherbourg,embark_town_Queenstown,embark_town_Southampton
0,-0.666007,-0.983747,-1.40498,-0.469216,0.748622,2.467844,0.426882,-0.90581,1.187705,0.081116,-0.179173,-0.123896


❓ What are the top 2 features driving chances of survival according to your model?  
Fill the `top_2_features` list below with the name of these features

In [34]:
top_2_features = ["sex_female", "who_child"]

#### 🧪 Test your code below

In [35]:
from nbresult import ChallengeResult
result = ChallengeResult('ridge', top_2 = top_2_features)
result.write()
print(result.check())


============================= test session starts ==============================
platform linux -- Python 3.10.6, pytest-7.1.3, pluggy-1.0.0 -- /home/leomichel/.pyenv/versions/lewagon/bin/python3
cachedir: .pytest_cache
rootdir: /home/leomichel/code/LeonMichel96/data-regularization/tests
plugins: anyio-3.6.2, asyncio-0.19.0
asyncio: mode=strict
collecting ... collected 1 item

test_ridge.py::TestRidge::test_top2 PASSED                               [100%]

============================== 1 passed in 0.03s ===============================


💯 You can commit your code:

git add tests/ridge.pickle

git commit -m 'Completed ridge step'

git push origin master



## 2. Logistic Regression with an L1 penalty

This time, we'll use a logistic model whose log-loss has been penalized with a **L1** term to **filter-out the less important features**.  
This is the "classification" equivalent to the **Lasso** regressor

❓ Instantiate a **strongly regularized** `LogisticRegression` and rank its features by importance

In [36]:
model_3 = LogisticRegression(penalty='l1', max_iter=1000, solver='liblinear').fit(X_train, y_train)

In [37]:
model_3.coef_

array([[-1.51924129, -0.95644661, -1.75639604, -0.02013181,  0.40066235,
         2.54054018,  0.02926436, -0.51784786,  1.25272276,  0.13771721,
         0.        ,  0.        ]])

In [38]:
coef_3 = pd.DataFrame(model_3.coef_, columns = X.columns)
coef_3

,pclass,age,sibsp,parch,fare,sex_female,class_First,class_Third,who_child,embark_town_Cherbourg,embark_town_Queenstown,embark_town_Southampton
0,-1.519241,-0.956447,-1.756396,-0.020132,0.400662,2.54054,0.029264,-0.517848,1.252723,0.137717,0.0,0.0


❓ What are the features that have absolutely no impact on chances of survival, according to your L1 model?  
Fill the `zero_impact_features` list below with the name of these features; you may have to add elements to the list.

- Do you notice how some of them were "highly important" according to the non-regularized model? 
- From now on, we will always regularize our linear models!

In [39]:
zero_impact_features = ["embark_town_Queenstown", "embark_town_Southampton", "parch", "class_First"]

#### 🧪 Test your code below

In [40]:
from nbresult import ChallengeResult
result = ChallengeResult('lasso', zero_impact_features = zero_impact_features)
result.write()
print(result.check())


============================= test session starts ==============================
platform linux -- Python 3.10.6, pytest-7.1.3, pluggy-1.0.0 -- /home/leomichel/.pyenv/versions/lewagon/bin/python3
cachedir: .pytest_cache
rootdir: /home/leomichel/code/LeonMichel96/data-regularization/tests
plugins: anyio-3.6.2, asyncio-0.19.0
asyncio: mode=strict
collecting ... collected 1 item

test_lasso.py::TestLasso::test_zero_impact PASSED                        [100%]

============================== 1 passed in 0.03s ===============================


💯 You can commit your code:

git add tests/lasso.pickle

git commit -m 'Completed lasso step'

git push origin master



**🏁 Congratulation! Don't forget to commit and push your notebook**